In [21]:
import pandas as pd
import string
import re
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import WhitespaceTokenizer

In [17]:
train = pd.read_csv('data/evo_train.csv.gz', compression='gzip', index_col='id')
test = pd.read_csv('data/evo_test.csv.gz', compression='gzip', index_col='id')

In [24]:
def parse_sentence(sentence, tokenizer, morph):
    return " ".join([morph.parse(w)[0].normal_form for w in tokenizer.tokenize(sentence)])

def parse(series, tokenizer = WhitespaceTokenizer(), morph = MorphAnalyzer()):
    return series.apply(lambda row: parse_sentence(row, tokenizer, morph))

def prepare(data):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    series = data['NAME'].str.lower().str.replace(',', '').replace(regex, " ").str.replace('\d+', ' NUMBER ')
    return parse(series)

In [26]:
train_names = prepare(train)
test_names = prepare(test)

In [31]:
train["NAME"] = train_names
test["NAME"] = test_names

In [32]:
train.to_csv('data/evo_train_clean.csv.gz', compression='gzip')
test.to_csv('data/evo_test_clean.csv.gz', compression='gzip')